In [27]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
import os, cv2, h5py
import torch
import torchvision
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import torch.nn.functional as F
import torch.utils.data as utils

from sklearn.metrics import confusion_matrix
from tqdm import tqdm_notebook as tqdm
from copy import deepcopy
from time import time
from mpl_toolkits import mplot3d
from torchvision import datasets, transforms
from torch import nn, optim
from torch import autograd

In [4]:
X = h5py.File("./drive/My Drive/data/X_train.h5", "r")["features"][:]
Y = pd.read_csv("./drive/My Drive/data/y_train.csv").as_matrix()[:, 1].squeeze()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [0]:
n = len(X)
x_train = X[:int(0.7*n), 11:]
y_train = Y[:int(0.7*n)]
x_test  = X[int(0.7*n):, 11:]
y_test  = Y[int(0.7*n):]
x_train = x_train[:, ::2]
x_test  = x_test[:, ::2]
x_train = np.array([[j for i, j in enumerate(eeg[10:]) if i % 6] for eeg in x_train])

x_train_meta = X[:int(0.7*n), :11]
x_test_meta = X[int(0.7*n):, :11]

In [0]:
x_train = np.clip(x_train, -205, 205)
x_test  = np.clip(x_test, -205, 205)

In [0]:
x_train = x_train / 205
x_test  = x_test / 205

In [0]:
maxi = np.max(x_train_meta, axis = 0)
mini = np.min(x_train_meta, axis = 0)

In [0]:
x_train_meta = (x_train_meta - maxi) / (mini - maxi)
x_test_meta  = (x_test_meta - maxi) / (mini - maxi)

In [0]:
n = len(X)
x_train = X[:, 11:]
y_train = Y[:]
x_train = x_train[:, ::2]
x_train = np.array([[j for i, j in enumerate(eeg[10:]) if i % 6] for eeg in x_train])
x_train_meta = X[:, :11]

In [0]:
x_train = np.clip(x_train, -205, 205)
x_train = x_train / 205
maxi = np.max(x_train_meta, axis = 0)
mini = np.min(x_train_meta, axis = 0)
x_train_meta = (x_train_meta - maxi) / (mini - maxi)

In [0]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv1d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

def conv5x5(in_planes, out_planes, stride=1):
    return nn.Conv1d(in_planes, out_planes, kernel_size=5, stride=stride,
                     padding=1, bias=False)

def conv7x7(in_planes, out_planes, stride=1):
    return nn.Conv1d(in_planes, out_planes, kernel_size=7, stride=stride,
                     padding=1, bias=False)



class BasicBlock3x3(nn.Module):
    expansion = 1

    def __init__(self, inplanes3, planes, stride=1, downsample=None):
        super(BasicBlock3x3, self).__init__()
        self.conv1 = conv3x3(inplanes3, planes, stride)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class BasicBlock5x5(nn.Module):
    expansion = 1

    def __init__(self, inplanes5, planes, stride=1, downsample=None):
        super(BasicBlock5x5, self).__init__()
        self.conv1 = conv5x5(inplanes5, planes, stride)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv5x5(planes, planes)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        d = residual.shape[2] - out.shape[2]
        out1 = residual[:,:,0:-d] + out
        out1 = self.relu(out1)
        # out += residual

        return out1



class BasicBlock7x7(nn.Module):
    expansion = 1

    def __init__(self, inplanes7, planes, stride=1, downsample=None):
        super(BasicBlock7x7, self).__init__()
        self.conv1 = conv7x7(inplanes7, planes, stride)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv7x7(planes, planes)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        d = residual.shape[2] - out.shape[2]
        out1 = residual[:, :, 0:-d] + out
        out1 = self.relu(out1)
        # out += residual

        return out1




class MSResNet(nn.Module):
    def __init__(self, input_channel, layers=[1, 1, 1, 1], num_classes=10):
        self.inplanes3 = 64
        self.inplanes5 = 64
        self.inplanes7 = 64

        super(MSResNet, self).__init__()

        self.conv1 = nn.Conv1d(input_channel, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.layer3x3_1 = self._make_layer3(BasicBlock3x3, 64, layers[0], stride=2)
        self.layer3x3_2 = self._make_layer3(BasicBlock3x3, 128, layers[1], stride=2)
        self.layer3x3_3 = self._make_layer3(BasicBlock3x3, 256, layers[2], stride=2)
        # self.layer3x3_4 = self._make_layer3(BasicBlock3x3, 512, layers[3], stride=2)

        # maxplooing kernel size: 16, 11, 6
        self.maxpool3 = nn.AvgPool1d(kernel_size=16, stride=1, padding=0)


        self.layer5x5_1 = self._make_layer5(BasicBlock5x5, 64, layers[0], stride=2)
        self.layer5x5_2 = self._make_layer5(BasicBlock5x5, 128, layers[1], stride=2)
        self.layer5x5_3 = self._make_layer5(BasicBlock5x5, 256, layers[2], stride=2)
        # self.layer5x5_4 = self._make_layer5(BasicBlock5x5, 512, layers[3], stride=2)
        self.maxpool5 = nn.AvgPool1d(kernel_size=11, stride=1, padding=0)


        self.layer7x7_1 = self._make_layer7(BasicBlock7x7, 64, layers[0], stride=2)
        self.layer7x7_2 = self._make_layer7(BasicBlock7x7, 128, layers[1], stride=2)
        self.layer7x7_3 = self._make_layer7(BasicBlock7x7, 256, layers[2], stride=2)
        # self.layer7x7_4 = self._make_layer7(BasicBlock7x7, 512, layers[3], stride=2)
        self.maxpool7 = nn.AvgPool1d(kernel_size=6, stride=1, padding=0)

        # self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(256*3, 128)

        # todo: modify the initialization
        # for m in self.modules():
        #     if isinstance(m, nn.Conv1d):
        #         n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        #         m.weight.data.normal_(0, math.sqrt(2. / n))
        #     elif isinstance(m, nn.BatchNorm1d):
        #         m.weight.data.fill_(1)
        #         m.bias.data.zero_()

    def _make_layer3(self, block, planes, blocks, stride=2):
        downsample = None
        if stride != 1 or self.inplanes3 != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes3, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes3, planes, stride, downsample))
        self.inplanes3 = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes3, planes))

        return nn.Sequential(*layers)

    def _make_layer5(self, block, planes, blocks, stride=2):
        downsample = None
        if stride != 1 or self.inplanes5 != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes5, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes5, planes, stride, downsample))
        self.inplanes5 = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes5, planes))

        return nn.Sequential(*layers)


    def _make_layer7(self, block, planes, blocks, stride=2):
        downsample = None
        if stride != 1 or self.inplanes7 != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes7, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes7, planes, stride, downsample))
        self.inplanes7 = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes7, planes))

        return nn.Sequential(*layers)

    def forward(self, x0):
        x0 = self.conv1(x0)
        x0 = self.bn1(x0)
        x0 = self.relu(x0)
        x0 = self.maxpool(x0)
        x = self.layer3x3_1(x0)
        x = self.layer3x3_2(x)
        x = self.layer3x3_3(x)
        # x = self.layer3x3_4(x)
        x = self.maxpool3(x)

        y = self.layer5x5_1(x0)
        y = self.layer5x5_2(y)
        y = self.layer5x5_3(y)
        # y = self.layer5x5_4(y)
        y = self.maxpool5(y)

        z = self.layer7x7_1(x0)
        z = self.layer7x7_2(z)
        z = self.layer7x7_3(z)
        # z = self.layer7x7_4(z)
        z = self.maxpool7(z)

        out = torch.cat([x, y, z], dim=1)
        
        out = out.squeeze()
        # out = self.drop(out)
        out1 = self.fc(out)

        return out1

In [0]:
def get_distance(data):
    dot_product = torch.matmul(data, data.transpose(1, 0))
    diag        = torch.diag(dot_product)
    distances   = diag.view((1, diag.shape[0])) - 2*dot_product + diag.view((diag.shape[0], 1))
    distances   = torch.max(distances, torch.tensor([0]).float().cuda())
    mask        = torch.eq(distances, torch.zeros(1).cuda()).float().cuda()
    distances   = distances + mask * 1e-16
    distances   = torch.sqrt(distances)
    distances   = distances * (1.0 - mask)
    return distances

In [0]:
def get_mask_triplet(labels):
    hot = F.one_hot(labels, 3).float()
    positive_mask = torch.matmul(hot, hot.transpose(0, 1))
    size = positive_mask.shape[0]
    positive_mask[range(size), range(size)] = torch.zeros(positive_mask.shape[0]).long().cuda()
    negative_mask = torch.matmul(torch.ones(hot.shape).long().cuda() - hot, hot.transpose(0, 1))
    mask = torch.matmul(positive_mask.view((size, size, 1)), negative_mask.view((size, 1, size)))
    return mask

In [0]:
def triplet_loss(data, labels, m):
    distances = get_distance(data)
    size = distances.shape[0]
    delta_p   = distances.view((size, size, 1)).repeat((1, 1, size)) + m
    delta_n   = distances.view((size, 1, size)).repeat((1, size, 1))
    triplet_loss = delta_p - delta_n
    mask_hard = torch.gt(delta_p, delta_n).float()
    mask      = get_mask_triplet(labels).float() * mask_hard
    triplet_loss = triplet_loss * mask
    triplet_loss = torch.max(triplet_loss, torch.zeros(1).float().cuda())
    
    nb_of_tensor = torch.gt(triplet_loss, torch.tensor([1e-16]).float().cuda())
    nb_of_tensor = torch.sum(nb_of_tensor)
    triplet_loss = torch.sum(triplet_loss)
    return triplet_loss / (nb_of_tensor + torch.tensor([1e-16]).float().cuda())

In [0]:
def augment(serie):
    noise = torch.randn(serie.shape[0], 1, serie.shape[-1]).float().cuda() * 0.005
    serie = noise + serie
    coins = torch.gt(torch.randn((serie.shape[0], 1, 1)).float().cuda(), torch.tensor([0]).float().cuda()).float().cuda() * 2 - 1
    serie = serie * coins
    return serie

In [0]:
x_train = torch.tensor(x_train).float().cuda()
x_test  = torch.tensor(x_test).float().cuda()
y_train = torch.tensor(y_train).long().cuda()
y_test  = torch.tensor(y_test).long().cuda()
x_train_meta = torch.tensor(x_train_meta).float().cuda()
x_test_meta = torch.tensor(x_test_meta).float().cuda()

In [0]:
x_train = x_train.view((-1, 1, 512))
x_test  = x_test.view((-1, 1, 512))

In [0]:
train_data = utils.TensorDataset(x_train, x_train_meta, y_train)
test_data  = utils.TensorDataset(x_test, x_test_meta, y_test)

In [0]:
train_loader = utils.DataLoader(train_data, batch_size=32, shuffle=True)
test_loader  = utils.DataLoader(test_data, batch_size=32, shuffle=True)

In [0]:
x_train = torch.tensor(x_train).float().cuda()
y_train = torch.tensor(y_train).long().cuda()
x_train_meta = torch.tensor(x_train_meta).float().cuda()
x_train = x_train.view((-1, 1, 512))
train_data = utils.TensorDataset(x_train, x_train_meta, y_train)
train_loader = utils.DataLoader(train_data, batch_size=32, shuffle=True)

In [0]:
model = MSResNet(1).cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
epochs = 10
best_loss = 1000
for e in range(epochs):
    running_loss = 0
    test_loss    = 0
    for i, (x, _, y) in enumerate(train_loader):
        x = augment(x)
        optimizer.zero_grad()
        output = model(x)
        loss   = triplet_loss(output, y, torch.tensor([1.]).float().cuda())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if not i % 500:
          print('Current loss : {0:.2f}'.format(loss.item()))
   
torch.save(model, '/content/drive/My Drive/data/model_emb.pt')


Current loss : 1.59
Current loss : 0.96
Current loss : 1.05
Current loss : 0.96
Current loss : 1.05
Current loss : 0.95
Current loss : 1.06
Current loss : 1.04
Current loss : 0.99
Current loss : 0.85
Current loss : 1.05
Current loss : 0.98
Current loss : 0.89
Current loss : 1.00
Current loss : 0.98
Current loss : 0.88
Current loss : 1.00
Current loss : 1.04
Current loss : 1.07
Current loss : 1.01
Current loss : 0.93
Current loss : 1.03
Current loss : 1.04
Current loss : 1.03
Current loss : 1.02
Current loss : 0.98
Current loss : 1.08
Current loss : 1.09
Current loss : 0.96
Current loss : 1.02
Current loss : 1.01
Current loss : 1.04
Current loss : 1.05
Current loss : 0.98
Current loss : 0.80
Current loss : 1.06
Current loss : 1.03
Current loss : 0.97
Current loss : 1.03
Current loss : 0.99
Current loss : 1.13
Current loss : 1.10
Current loss : 0.99
Current loss : 0.99
Current loss : 0.93
Current loss : 1.09
Current loss : 0.97
Current loss : 1.00
Current loss : 0.86
Current loss : 1.08


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type MSResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv1d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm1d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/di

In [22]:
model = torch.load('/content/drive/My Drive/data/model_emb.pt')
model.eval()

MSResNet(
  (conv1): Conv1d(1, 64, kernel_size=(7,), stride=(2,), padding=(3,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer3x3_1): Sequential(
    (0): BasicBlock3x3(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(2,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv1d(64, 64, kernel_size=(1,), stride=(2,), bias=False)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (layer3x3_2): Sequential(
    (0): BasicB

In [0]:
class Net(nn.Module):

  def __init__(self):
        super(Net, self).__init__()
        self.fc1   = nn.Linear(128+11, 128)
        self.fc2   = nn.Linear(128, 32)
        self.fc3   = nn.Linear(32, 3)
        self.drop1 = nn.Dropout(0.3)
        self.drop2 = nn.Dropout(0.2)
        
  def forward(self, x, eeg):
    x = torch.cat([x, eeg], -1)
    x = F.relu(self.fc1(x))
    x = self.drop1(x)
    x = F.relu(self.fc2(x))
    x = self.drop2(x)
    x = self.fc3(x)
    return F.log_softmax(x, dim=1)

In [0]:
net = Net().cuda()
loss_function = nn.NLLLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [28]:
epochs = 9
best_loss = 1000000
for e in range(epochs):
    running_loss = 0
    test_loss    = 0
    for i, (x, meta, y) in enumerate(train_loader):
        x = augment(x)
        optimizer.zero_grad()
        embedding = model(x)
        
        output = net(meta, embedding)
        loss = loss_function(output, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if not i % 500:
          print('Current loss : {0:.2f}'.format(loss.item()))
    if running_loss < best_loss:
      torch.save(model, '/content/drive/My Drive/data/netyouhou.pt')
      best_loss = running_loss

Current loss : 0.98
Current loss : 0.89
Current loss : 0.99
Current loss : 1.07
Current loss : 0.89
Current loss : 0.96
Current loss : 0.99
Current loss : 1.08
Current loss : 0.99
Current loss : 0.99
Current loss : 0.85
Current loss : 0.83
Current loss : 0.98
Current loss : 0.89
Current loss : 0.99
Current loss : 0.99
Current loss : 1.06


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type MSResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv1d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm1d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/di

Current loss : 1.19
Current loss : 0.96
Current loss : 1.05
Current loss : 0.95
Current loss : 1.05
Current loss : 0.84
Current loss : 0.99
Current loss : 1.08
Current loss : 0.97
Current loss : 0.89
Current loss : 0.92
Current loss : 0.87
Current loss : 0.99
Current loss : 0.98
Current loss : 0.92
Current loss : 0.85
Current loss : 1.09
Current loss : 0.92
Current loss : 0.89
Current loss : 1.09
Current loss : 0.92
Current loss : 1.02
Current loss : 0.93
Current loss : 0.98
Current loss : 1.00
Current loss : 1.04
Current loss : 0.90
Current loss : 1.04
Current loss : 0.88
Current loss : 0.86
Current loss : 0.93
Current loss : 1.10
Current loss : 0.97
Current loss : 0.96
Current loss : 1.00
Current loss : 0.95
Current loss : 0.93
Current loss : 0.98
Current loss : 1.10
Current loss : 1.02
Current loss : 1.00
Current loss : 1.04
Current loss : 1.17
Current loss : 0.93
Current loss : 1.07
Current loss : 0.97
Current loss : 0.98
Current loss : 0.90
Current loss : 0.98
Current loss : 0.86


In [0]:
output

tensor([[-7.9879e+00, -1.0588e+00, -4.2652e-01],
        [-4.8600e+01,  0.0000e+00, -6.0397e+01],
        [-6.3864e+01,  0.0000e+00, -6.3916e+01],
        [ 0.0000e+00, -3.1545e+02, -1.9885e+02],
        [ 0.0000e+00, -2.8043e+02, -5.0497e+01],
        [ 0.0000e+00, -3.9889e+01, -1.0874e+02],
        [ 0.0000e+00, -2.2874e+01, -2.9230e+01],
        [ 0.0000e+00, -1.2878e+02, -1.5468e+02],
        [ 0.0000e+00, -5.8744e+02, -3.8296e+02],
        [ 0.0000e+00, -2.8867e+02, -5.0404e+01],
        [-9.3705e+01, -2.7942e+02,  0.0000e+00],
        [-1.7382e+01, -9.1674e+01,  0.0000e+00],
        [-2.3556e+01,  0.0000e+00, -7.1566e+01],
        [-6.1738e+01, -5.3609e+01,  0.0000e+00],
        [ 0.0000e+00, -3.7475e+02, -5.5885e+01],
        [-6.8465e+02, -3.3675e+02,  0.0000e+00],
        [-3.8895e+01, -7.6294e-06, -1.2345e+01],
        [-1.4709e+02,  0.0000e+00, -3.1391e+01],
        [-8.2675e+02,  0.0000e+00, -5.3040e+02],
        [-1.3842e+02, -6.9520e+01,  0.0000e+00],
        [-2.7895e+01

In [0]:
X = h5py.File("./drive/My Drive/data/X_test_eFVIB85.h5", "r")["features"][:]

In [0]:
x_test = X[:, 11:]
x_test_meta = X[:, :11]

x_test  = x_test[:, ::2]
x_test  = np.array([[j for i, j in enumerate(eeg[10:]) if i % 6] for eeg in x_test])

In [0]:
x_test  = np.clip(x_test, -205, 205)
x_test = x_test / 205

In [0]:
x_test_meta  = (x_test_meta - maxi) / (mini - maxi)

In [0]:
x_test  = torch.tensor(x_test).float().cuda()
x_test_meta = torch.tensor(x_test_meta).float().cuda()
x_test  = x_test.view((-1, 1, 512))

In [0]:
test_data  = utils.TensorDataset(x_test, x_test_meta)

In [0]:
test_loader  = utils.DataLoader(test_data, batch_size=32, shuffle=False)

In [36]:
net.eval()

Net(
  (fc1): Linear(in_features=139, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=3, bias=True)
  (drop1): Dropout(p=0.3, inplace=False)
  (drop2): Dropout(p=0.2, inplace=False)
)

In [0]:
out = []
for x, meta in test_loader:
  embedding = model(x)
  output = net(meta, embedding)
  out.extend(output.cpu().detach())

In [38]:
out = np.array(out)

ValueError: ignored

In [0]:
out = np.array([i.cpu().detach().numpy() for i in out])

In [40]:
out.shape

(238366, 3)

In [0]:
y_pred = np.argmax(out, axis = 1)

In [0]:
len(y_pred)

238366

In [0]:
df = pd.DataFrame()

In [0]:
df['id'] = range(238366)

In [0]:
df['label'] = y_pred

In [0]:
df.to_csv('./drive/My Drive/data/cc.csv')